In [40]:
# Importing necessary Libraries
import sqlite3
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt


In [41]:
# Create your connection.
cnx = sqlite3.connect('database.sqlite')
df = pd.read_sql_query("SELECT * FROM Player_Attributes", cnx)

In [42]:
df.head()

,id,player_fifa_api_id,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,1,218353,505942,2016-02-18 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
1,2,218353,505942,2015-11-19 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
2,3,218353,505942,2015-09-21 00:00:00,62.0,66.0,right,medium,medium,49.0,...,54.0,48.0,65.0,66.0,69.0,6.0,11.0,10.0,8.0,8.0
3,4,218353,505942,2015-03-20 00:00:00,61.0,65.0,right,medium,medium,48.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
4,5,218353,505942,2007-02-22 00:00:00,61.0,65.0,right,medium,medium,48.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0


In [43]:
df['overall_rating'].unique()

array([67., 62., 61., 74., 73., 71., 70., 69., 68., 65., 64., 54., 51.,
       52., 47., 53., 66., 59., 75., 72., 76., 78., 77., 79., 60., 80.,
       81., 82., 84., 48., 63., 83., 55., 58., 50., 56., nan, 49., 57.,
       42., 46., 45., 85., 44., 86., 89., 87., 88., 91., 40., 90., 41.,
       43., 38., 93., 92., 39., 33., 36., 37., 35., 94.])

In [44]:
df.columns

Index(['id', 'player_fifa_api_id', 'player_api_id', 'date', 'overall_rating',
       'potential', 'preferred_foot', 'attacking_work_rate',
       'defensive_work_rate', 'crossing', 'finishing', 'heading_accuracy',
       'short_passing', 'volleys', 'dribbling', 'curve', 'free_kick_accuracy',
       'long_passing', 'ball_control', 'acceleration', 'sprint_speed',
       'agility', 'reactions', 'balance', 'shot_power', 'jumping', 'stamina',
       'strength', 'long_shots', 'aggression', 'interceptions', 'positioning',
       'vision', 'penalties', 'marking', 'standing_tackle', 'sliding_tackle',
       'gk_diving', 'gk_handling', 'gk_kicking', 'gk_positioning',
       'gk_reflexes'],
      dtype='object')

In [45]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 183978 entries, 0 to 183977
Data columns (total 42 columns):
id                     183978 non-null int64
player_fifa_api_id     183978 non-null int64
player_api_id          183978 non-null int64
date                   183978 non-null object
overall_rating         183142 non-null float64
potential              183142 non-null float64
preferred_foot         183142 non-null object
attacking_work_rate    180748 non-null object
defensive_work_rate    183142 non-null object
crossing               183142 non-null float64
finishing              183142 non-null float64
heading_accuracy       183142 non-null float64
short_passing          183142 non-null float64
volleys                181265 non-null float64
dribbling              183142 non-null float64
curve                  181265 non-null float64
free_kick_accuracy     183142 non-null float64
long_passing           183142 non-null float64
ball_control           183142 non-null float64
accele

We can see there are some objects in the dataset.

Also non necessary features in the dataset

In [46]:
df.describe(include='all')

,id,player_fifa_api_id,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
count,183978.00000,183978.000000,183978.000000,183978,183142.000000,183142.000000,183142,180748,183142,183142.000000,...,181265.000000,183142.000000,183142.000000,183142.000000,181265.000000,183142.000000,183142.000000,183142.000000,183142.000000,183142.000000
unique,NaN,NaN,NaN,197,NaN,NaN,2,8,19,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,NaN,NaN,2007-02-22 00:00:00,NaN,NaN,right,medium,medium,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,NaN,NaN,11794,NaN,NaN,138409,125070,130846,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,91989.50000,165671.524291,135900.617324,NaN,68.600015,73.460353,NaN,NaN,NaN,55.086883,...,57.873550,55.003986,46.772242,50.351257,48.001462,14.704393,16.063612,20.998362,16.132154,16.441439
std,53110.01825,53851.094769,136927.840510,NaN,7.041139,6.592271,NaN,NaN,NaN,17.242135,...,15.144086,15.546519,21.227667,21.483706,21.598778,16.865467,15.867382,21.452980,16.099175,17.198155
min,1.00000,2.000000,2625.000000,NaN,33.000000,39.000000,NaN,NaN,NaN,1.000000,...,1.000000,2.000000,1.000000,1.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,45995.25000,155798.000000,34763.000000,NaN,64.000000,69.000000,NaN,NaN,NaN,45.000000,...,49.000000,45.000000,25.000000,29.000000,25.000000,7.000000,8.000000,8.000000,8.000000,8.000000
50%,91989.50000,183488.000000,77741.000000,NaN,69.000000,74.000000,NaN,NaN,NaN,59.000000,...,60.000000,57.000000,50.000000,56.000000,53.000000,10.000000,11.000000,12.000000,11.000000,11.000000
75%,137983.75000,199848.000000,191080.000000,NaN,73.000000,78.000000,NaN,NaN,NaN,68.000000,...,69.000000,67.000000,66.000000,69.000000,67.000000,13.000000,15.000000,15.000000,15.000000,15.000000


In [47]:
# Shape of Dataset
df.shape

(183978, 42)

In [48]:
# Check any Nan values
df.isna().sum()

id                        0
player_fifa_api_id        0
player_api_id             0
date                      0
overall_rating          836
potential               836
preferred_foot          836
attacking_work_rate    3230
defensive_work_rate     836
crossing                836
finishing               836
heading_accuracy        836
short_passing           836
volleys                2713
dribbling               836
curve                  2713
free_kick_accuracy      836
long_passing            836
ball_control            836
acceleration            836
sprint_speed            836
agility                2713
reactions               836
balance                2713
shot_power              836
jumping                2713
stamina                 836
strength                836
long_shots              836
aggression              836
interceptions           836
positioning             836
vision                 2713
penalties               836
marking                 836
standing_tackle     

In [49]:
df['overall_rating'].unique()

array([67., 62., 61., 74., 73., 71., 70., 69., 68., 65., 64., 54., 51.,
       52., 47., 53., 66., 59., 75., 72., 76., 78., 77., 79., 60., 80.,
       81., 82., 84., 48., 63., 83., 55., 58., 50., 56., nan, 49., 57.,
       42., 46., 45., 85., 44., 86., 89., 87., 88., 91., 40., 90., 41.,
       43., 38., 93., 92., 39., 33., 36., 37., 35., 94.])

In [50]:
df.isna().sum().count()

42

In [51]:
# Check any duplicates in dataset
df.duplicated().any()

False

In [52]:
# df[df[''].isna()].index.tolist()

In [53]:
# drop unwanted columns that are not required 
df = df.drop(['id','player_fifa_api_id','player_api_id','date'],axis=1)

In [54]:
df.columns

Index(['overall_rating', 'potential', 'preferred_foot', 'attacking_work_rate',
       'defensive_work_rate', 'crossing', 'finishing', 'heading_accuracy',
       'short_passing', 'volleys', 'dribbling', 'curve', 'free_kick_accuracy',
       'long_passing', 'ball_control', 'acceleration', 'sprint_speed',
       'agility', 'reactions', 'balance', 'shot_power', 'jumping', 'stamina',
       'strength', 'long_shots', 'aggression', 'interceptions', 'positioning',
       'vision', 'penalties', 'marking', 'standing_tackle', 'sliding_tackle',
       'gk_diving', 'gk_handling', 'gk_kicking', 'gk_positioning',
       'gk_reflexes'],
      dtype='object')

In [55]:
df.isna().sum().count()

38

In [56]:
# df[df['overall_rating'].isna()] == df[df['potential'].isna()] == df[df['preferred_foot'].isna()] == df[df['attacking_work_rate'].isna()]

In [57]:
df['overall_rating'].unique()

array([67., 62., 61., 74., 73., 71., 70., 69., 68., 65., 64., 54., 51.,
       52., 47., 53., 66., 59., 75., 72., 76., 78., 77., 79., 60., 80.,
       81., 82., 84., 48., 63., 83., 55., 58., 50., 56., nan, 49., 57.,
       42., 46., 45., 85., 44., 86., 89., 87., 88., 91., 40., 90., 41.,
       43., 38., 93., 92., 39., 33., 36., 37., 35., 94.])

In [58]:
# drop Common Nan 
df.dropna(subset=['overall_rating', 'potential', 'preferred_foot', 'attacking_work_rate',
       'defensive_work_rate', 'crossing', 'finishing', 'heading_accuracy',
       'short_passing', 'volleys', 'dribbling', 'curve', 'free_kick_accuracy',
       'long_passing', 'ball_control', 'acceleration', 'sprint_speed',
       'agility', 'reactions', 'balance', 'shot_power', 'jumping', 'stamina',
       'strength', 'long_shots', 'aggression', 'interceptions', 'positioning',
       'vision', 'penalties', 'marking', 'standing_tackle', 'sliding_tackle',
       'gk_diving', 'gk_handling', 'gk_kicking', 'gk_positioning',
       'gk_reflexes'], how='all', inplace=True)

In [59]:
df['overall_rating'].unique()

array([67., 62., 61., 74., 73., 71., 70., 69., 68., 65., 64., 54., 51.,
       52., 47., 53., 66., 59., 75., 72., 76., 78., 77., 79., 60., 80.,
       81., 82., 84., 48., 63., 83., 55., 58., 50., 56., 49., 57., 42.,
       46., 45., 85., 44., 86., 89., 87., 88., 91., 40., 90., 41., 43.,
       38., 93., 92., 39., 33., 36., 37., 35., 94.])

In [60]:
df.isna().sum()

overall_rating            0
potential                 0
preferred_foot            0
attacking_work_rate    2394
defensive_work_rate       0
crossing                  0
finishing                 0
heading_accuracy          0
short_passing             0
volleys                1877
dribbling                 0
curve                  1877
free_kick_accuracy        0
long_passing              0
ball_control              0
acceleration              0
sprint_speed              0
agility                1877
reactions                 0
balance                1877
shot_power                0
jumping                1877
stamina                   0
strength                  0
long_shots                0
aggression                0
interceptions             0
positioning               0
vision                 1877
penalties                 0
marking                   0
standing_tackle           0
sliding_tackle         1877
gk_diving                 0
gk_handling               0
gk_kicking          

In [61]:
df['attacking_work_rate'].unique()

array(['medium', 'high', None, 'low', 'None', 'le', 'norm', 'stoc', 'y'],
      dtype=object)

In [62]:
df['attacking_work_rate'].value_counts()

medium    125070
high       42823
low         8569
None        3639
norm         348
y            106
le           104
stoc          89
Name: attacking_work_rate, dtype: int64

In [63]:
df[df['attacking_work_rate']=='medium']=1
df[df['attacking_work_rate']=='high']=2
df[df['attacking_work_rate']=='none']=3
df[df['attacking_work_rate']=='low']=4
df[df['attacking_work_rate']=='le']=5
df[df['attacking_work_rate']=='norm']=6
df[df['attacking_work_rate']=='stoc']=7
df[df['attacking_work_rate']=='y']=8
df[df['attacking_work_rate']=='None']=8

In [64]:
df['attacking_work_rate'].value_counts()

1    125070
2     42823
4      8569
8      3745
6       348
5       104
7        89
Name: attacking_work_rate, dtype: int64

In [65]:
df['overall_rating'].unique()

array([ 1.,  2., 64., 66., 68., 70., 62., 61., 73., 77., 78., 56., 72.,
       59.,  4.,  8.,  5., 60., 71.,  6., 49., 63., 81., 83., 84., 74.,
       53., 46., 79., 58., 65., 69., 67., 80., 82., 88., 55.,  7., 76.,
       54., 44., 41., 57., 52., 75., 50., 48., 47., 85., 86., 91., 42.,
       51., 87., 45.])

In [66]:
df['attacking_work_rate'].fillna((df['attacking_work_rate'].median()), inplace=True)

In [67]:
df['volleys'].fillna((df['volleys'].mean()), inplace=True)

In [68]:
df['curve'].fillna((df['curve'].mean()), inplace=True)

In [69]:
df['agility'].fillna((df['agility'].mean()), inplace=True)

In [70]:
df['balance'].fillna((df['balance'].mean()), inplace=True)

In [71]:
df['jumping'].fillna((df['jumping'].mean()), inplace=True)

In [72]:
df['vision'].fillna((df['vision'].mean()), inplace=True)

In [73]:
df['sliding_tackle'].fillna((df['sliding_tackle'].mean()), inplace=True)

In [74]:
df['overall_rating'].unique()

array([ 1.,  2., 64., 66., 68., 70., 62., 61., 73., 77., 78., 56., 72.,
       59.,  4.,  8.,  5., 60., 71.,  6., 49., 63., 81., 83., 84., 74.,
       53., 46., 79., 58., 65., 69., 67., 80., 82., 88., 55.,  7., 76.,
       54., 44., 41., 57., 52., 75., 50., 48., 47., 85., 86., 91., 42.,
       51., 87., 45.])

In [75]:
df.isna().sum()

overall_rating         0
potential              0
preferred_foot         0
attacking_work_rate    0
defensive_work_rate    0
crossing               0
finishing              0
heading_accuracy       0
short_passing          0
volleys                0
dribbling              0
curve                  0
free_kick_accuracy     0
long_passing           0
ball_control           0
acceleration           0
sprint_speed           0
agility                0
reactions              0
balance                0
shot_power             0
jumping                0
stamina                0
strength               0
long_shots             0
aggression             0
interceptions          0
positioning            0
vision                 0
penalties              0
marking                0
standing_tackle        0
sliding_tackle         0
gk_diving              0
gk_handling            0
gk_kicking             0
gk_positioning         0
gk_reflexes            0
dtype: int64

In [76]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 183142 entries, 0 to 183977
Data columns (total 38 columns):
overall_rating         183142 non-null float64
potential              183142 non-null float64
preferred_foot         183142 non-null object
attacking_work_rate    183142 non-null float64
defensive_work_rate    183142 non-null object
crossing               183142 non-null float64
finishing              183142 non-null float64
heading_accuracy       183142 non-null float64
short_passing          183142 non-null float64
volleys                183142 non-null float64
dribbling              183142 non-null float64
curve                  183142 non-null float64
free_kick_accuracy     183142 non-null float64
long_passing           183142 non-null float64
ball_control           183142 non-null float64
acceleration           183142 non-null float64
sprint_speed           183142 non-null float64
agility                183142 non-null float64
reactions              183142 non-null float6

In [77]:
df['preferred_foot'].value_counts()

1        125070
2         42823
4          8569
8          3745
right      1858
left        536
6           348
5           104
7            89
Name: preferred_foot, dtype: int64

In [78]:
df[df['preferred_foot']=='right']=3
df[df['preferred_foot']=='left']=9

In [79]:
df['defensive_work_rate'] = df['defensive_work_rate'].astype('float')

In [80]:
df['overall_rating'].unique()

array([1., 2., 3., 9., 4., 8., 5., 6., 7.])

In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 183142 entries, 0 to 183977
Data columns (total 38 columns):
overall_rating         183142 non-null float64
potential              183142 non-null float64
preferred_foot         183142 non-null int64
attacking_work_rate    183142 non-null float64
defensive_work_rate    183142 non-null float64
crossing               183142 non-null float64
finishing              183142 non-null float64
heading_accuracy       183142 non-null float64
short_passing          183142 non-null float64
volleys                183142 non-null float64
dribbling              183142 non-null float64
curve                  183142 non-null float64
free_kick_accuracy     183142 non-null float64
long_passing           183142 non-null float64
ball_control           183142 non-null float64
acceleration           183142 non-null float64
sprint_speed           183142 non-null float64
agility                183142 non-null float64
reactions              183142 non-null float6

In [36]:
# Linear Regression

In [37]:
#import matplotlib.pyplot as plt
#%matplotlib inline

In [38]:
# Data Visualisation
#fig, axs = plt.subplots(13, 3, sharey=True)
df.plot(kind='scatter', x='overall_rating', y='potential', figsize=(6, 6))
#df.plot(kind='scatter', x='overall_rating', y='preferred_foot', ax=axs[0][1])
#df.plot(kind='scatter', x='overall_rating', y='attacking_work_rate', ax=axs[0][2])
#df.plot(kind='scatter', x='overall_rating', y='defensive_work_rate', ax=axs[1][0])
#df.plot(kind='scatter', x='overall_rating', y='crossing', ax=axs[1][1])
#df.plot(kind='scatter', x='overall_rating', y='finishing', ax=axs[1][2])
#df.plot(kind='scatter', x='overall_rating', y='heading_accuracy', ax=axs[2][0])
#df.plot(kind='scatter', x='overall_rating', y='short_passing', ax=axs[2][1])
#df.plot(kind='scatter', x='overall_rating', y='volleys', ax=axs[2][2])
#df.plot(kind='scatter', x='overall_rating', y='dribbling', ax=axs[3][0])
#df.plot(kind='scatter', x='overall_rating', y='curve', ax=axs[3][1])
#df.plot(kind='scatter', x='overall_rating', y='free_kick_accuracy', ax=axs[3][2])
#df.plot(kind='scatter', x='overall_rating', y='long_passing', ax=axs[4][0])
#df.plot(kind='scatter', x='overall_rating', y='ball_control', ax=axs[4][1])
#df.plot(kind='scatter', x='overall_rating', y='acceleration', ax=axs[4][2])
#df.plot(kind='scatter', x='overall_rating', y='sprint_speed', ax=axs[5][0])
#df.plot(kind='scatter', x='overall_rating', y='agility', ax=axs[5][1])
#df.plot(kind='scatter', x='overall_rating', y='reactions', ax=axs[5][2])
#df.plot(kind='scatter', x='overall_rating', y='balance', ax=axs[6][0])
#df.plot(kind='scatter', x='overall_rating', y='jumping', ax=axs[6][1])
#df.plot(kind='scatter', x='overall_rating', y='stamina', ax=axs[6][2])
#df.plot(kind='scatter', x='overall_rating', y='strength', ax=axs[7][0])
#df.plot(kind='scatter', x='overall_rating', y='long_shots', ax=axs[7][1])
#df.plot(kind='scatter', x='overall_rating', y='aggression', ax=axs[7][2])
#df.plot(kind='scatter', x='overall_rating', y='interceptions', ax=axs[8][0])
#df.plot(kind='scatter', x='overall_rating', y='positioning', ax=axs[8][1])
#df.plot(kind='scatter', x='overall_rating', y='vision', ax=axs[8][2])
#df.plot(kind='scatter', x='overall_rating', y='penalties', ax=axs[9][0])
#df.plot(kind='scatter', x='overall_rating', y='marking', ax=axs[9][1])
#df.plot(kind='scatter', x='overall_rating', y='standing_tackle', ax=axs[9][2])
#df.plot(kind='scatter', x='overall_rating', y='sliding_tackle', ax=axs[10][0])
#df.plot(kind='scatter', x='overall_rating', y='gk_diving', ax=axs[10][1])
#df.plot(kind='scatter', x='overall_rating', y='gk_handling', ax=axs[10][2])
#df.plot(kind='scatter', x='overall_rating', y='gk_kicking', ax=axs[11][0])
#df.plot(kind='scatter', x='overall_rating', y='gk_positioning', ax=axs[11][1])
#df.plot(kind='scatter', x='overall_rating', y='gk_reflexes', ax=axs[11][2])
#df.plot(kind='scatter', x='overall_rating', y='shot_power', ax=axs[12][0])


In [39]:
df['overall_rating'].unique()

array([1., 2., 3., 9., 4., 8., 5., 6., 7.])